Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=10, penalty='l2').fit(train_dataset[:50000], np.argmax(train_labels[:50000],axis=1))

In [79]:
from sklearn.metrics import classification_report
print('Val report')
print(classification_report(y_pred=model.predict(valid_dataset), y_true=np.argmax(valid_labels, axis=1)))
print('Test report')
print(classification_report(y_pred=model.predict(test_dataset), y_true=np.argmax(test_labels, axis=1)))

Val report
             precision    recall  f1-score   support

          0       0.83      0.81      0.82      1000
          1       0.82      0.77      0.80      1000
          2       0.81      0.87      0.84      1000
          3       0.84      0.84      0.84      1000
          4       0.82      0.75      0.78      1000
          5       0.83      0.86      0.84      1000
          6       0.82      0.79      0.80      1000
          7       0.82      0.82      0.82      1000
          8       0.74      0.79      0.77      1000
          9       0.81      0.82      0.82      1000

avg / total       0.81      0.81      0.81     10000

Test report
             precision    recall  f1-score   support

          0       0.90      0.87      0.89      1000
          1       0.90      0.88      0.89      1000
          2       0.86      0.93      0.89      1000
          3       0.92      0.91      0.92      1000
          4       0.88      0.82      0.85      1000
          5       0

In [80]:
def nn_reg():
    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      tf_reg_c = tf.constant(0.001)

      # my hidden layer
      hidden_w = tf.Variable(
        tf.truncated_normal([image_size* image_size, 1024])
      )
      hidden_b = tf.Variable(tf.zeros([1024]))
      hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_w) + hidden_b)


      # Variables.
      weights = tf.Variable(
        tf.truncated_normal([1024, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(hidden, weights) + biases
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

      reg = tf.nn.l2_loss(hidden_w) + tf.nn.l2_loss(weights)
      loss+=tf_reg_c * reg

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      predict = lambda dataset: tf.nn.softmax(tf.matmul( tf.nn.relu(tf.matmul(dataset, hidden_w) + hidden_b), weights) + biases)

      valid_prediction = predict(tf_valid_dataset)
      test_prediction = predict(tf_test_dataset)

In [81]:
nn_reg()
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 714.850891
Minibatch accuracy: 15.6%
Validation accuracy: 28.0%
Minibatch loss at step 500: 192.646881
Minibatch accuracy: 75.0%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 115.967949
Minibatch accuracy: 74.2%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 68.169044
Minibatch accuracy: 80.9%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 40.997562
Minibatch accuracy: 86.3%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 25.002024
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Minibatch loss at step 3000: 15.464198
Minibatch accuracy: 84.0%
Validation accuracy: 86.1%
Test accuracy: 92.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [82]:
train_dataset_small = train_dataset[:1000]
train_labels_small = train_labels[:1000]

train_dataset_small.shape, train_labels_small.shape

((1000, 784), (1000, 10))

In [83]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
#     print(batch_data.shape)
#     print(batch_labels.shape)
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset : batch_data,
        tf_train_labels : batch_labels
    }
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 825.213501
Minibatch accuracy: 7.4%
Validation accuracy: 31.5%
Minibatch loss at step 500: 190.578598
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 115.663048
Minibatch accuracy: 99.6%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 70.121437
Minibatch accuracy: 100.0%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 42.569256
Minibatch accuracy: 99.6%
Validation accuracy: 79.9%
Minibatch loss at step 2500: 25.793873
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 15.644109
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Test accuracy: 86.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [84]:
def nn_dropout():
    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      tf_reg_c = tf.constant(0.001)

      # my hidden layer
      hidden_w = tf.Variable(
        tf.truncated_normal([image_size* image_size, 1024])
      )
      hidden_b = tf.Variable(tf.zeros([1024]))
      hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_w) + hidden_b)
      hidden = tf.nn.dropout(hidden, 0.5)


      # Variables.
      weights = tf.Variable(
        tf.truncated_normal([1024, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))

      # Training computation.
      logits = tf.matmul(hidden, weights) + biases
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

      reg = tf.nn.l2_loss(hidden_w) + tf.nn.l2_loss(weights)
      loss+=tf_reg_c * reg

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      predict = lambda dataset: tf.nn.softmax(tf.matmul( tf.nn.relu(tf.matmul(dataset, hidden_w) + hidden_b), weights) + biases)

      valid_prediction = predict(tf_valid_dataset)
      test_prediction = predict(tf_test_dataset)


In [85]:
num_steps = 3001
nn_dropout()
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
#     print(batch_data.shape)
#     print(batch_labels.shape)
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset : batch_data,
        tf_train_labels : batch_labels
    }
    _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 815.787231
Minibatch accuracy: 13.3%
Validation accuracy: 34.5%
Minibatch loss at step 500: 191.031906
Minibatch accuracy: 98.4%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 115.541931
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 70.085350
Minibatch accuracy: 99.6%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 42.503922
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 25.779585
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 15.637362
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Test accuracy: 86.6%


Без dropout мрежата е научила train dataset-a с по-бързо

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


https://en.wikipedia.org/wiki/Exponential_decay

In [142]:
def nn_3layerd():
    '''
    input -> hidden(1024)(relu) - > dropout(0.5) -> hidden(128)(relu) -> dropout(0.3) -> output(softmax)
    '''
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)

        tf_reg_c = tf.constant(0.001)

        # my hidden layer
        hidden_w = tf.Variable(
        tf.truncated_normal([image_size* image_size, 512])
        )
        hidden_b = tf.Variable(tf.zeros([512]))
        hidden = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_w) + hidden_b)
        hidden = tf.nn.dropout(hidden, 0.4)

        # hidden layer 2
        hidden2_w = tf.Variable(
        tf.truncated_normal([512, 256])
        )
        hidden2_b = tf.Variable(tf.zeros([256]))
        hidden2 = tf.nn.relu(tf.matmul(hidden, hidden2_w) + hidden2_b)
        hidden2 = tf.nn.dropout(hidden2, 0.3)

        # Variables.
        weights = tf.Variable(
        tf.truncated_normal([256, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        # Training computation.
        logits = tf.matmul(hidden2, weights) + biases
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

        reg = tf.nn.l2_loss(hidden_w) + tf.nn.l2_loss(hidden2_w) + tf.nn.l2_loss(weights)
        loss+=tf_reg_c * reg
        
        # Optimizer
#         global_step = tf.Variable(0)
#         start = 0.00001
#         lr = tf.train.exponential_decay(start, global_step, 600, 0.85)
        optimizer = tf.train.AdamOptimizer().minimize(loss)
        
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)

        predict = lambda dataset: tf.nn.softmax(tf.matmul( tf.nn.relu(tf.matmul( tf.nn.relu(tf.matmul(dataset, hidden_w) + hidden_b), hidden2_w) + hidden2_b) , weights) + biases)

        valid_prediction = predict(tf_valid_dataset)
        test_prediction = predict(tf_test_dataset)


In [144]:
num_steps = 25000
nn_3layerd()
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        #     print(batch_data.shape)
        #     print(batch_labels.shape)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {
            tf_train_dataset : batch_data,
            tf_train_labels : batch_labels
        }
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 779.982910
Minibatch accuracy: 5.9%
Validation accuracy: 28.4%
Minibatch loss at step 500: 197.421295
Minibatch accuracy: 72.3%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 115.119392
Minibatch accuracy: 76.2%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 68.590149
Minibatch accuracy: 76.2%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.190937
Minibatch accuracy: 85.2%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 25.031212
Minibatch accuracy: 85.5%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.487282
Minibatch accuracy: 84.8%
Validation accuracy: 86.3%
Minibatch loss at step 3500: 9.546672
Minibatch accuracy: 85.9%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 5.993134
Minibatch accuracy: 86.7%
Validation accuracy: 87.3%
Minibatch loss at step 4500: 3.831416
Minibatch accuracy: 89.1%
Validation accuracy: 87.6%
Minibatch loss at step 5000: 2.481787
Minibatch accuracy: 87.9%
Validatio